In [ ]:
# Install the required libraries if running in Kaggle
!pip install googletrans==4.0.0-rc1
!pip install python-Levenshtein

# Import required libraries
import pandas as pd
from googletrans import Translator
import re
import Levenshtein

# Load the Excel file
file_path = '/kaggle/input/phy-name-id/extracted_doctor_names (2) (1).xlsx'
df = pd.read_excel(file_path)

# Initialize the translator
translator = Translator()

# Function to check if a string contains Bangla characters
def contains_bangla(text):
    bangla_pattern = re.compile(r'[\u0980-\u09FF]')
    return bool(bangla_pattern.search(text))

# Function to translate Bangla names to English
def translate_name(name):
    name = str(name)  # Ensure the name is a string
    if contains_bangla(name):
        try:
            translated = translator.translate(name, src='bn', dest='en')
            return translated.text.upper()  # Convert to uppercase
        except Exception as e:
            print(f"Error translating {name}: {e}")
            return None
    else:
        return name.upper()  # Convert to uppercase

# Apply translation to the doctor_name column
df['translated'] = df['doctor_name'].apply(translate_name)

# Function to calculate similarity percentage
def similarity_percentage(str1, str2):
    return Levenshtein.ratio(str1, str2) * 100

# Function to find matched names between translated name and PHY_NM column based on 80% similarity
def find_matched_name(row, phy_nm_list):
    if pd.isnull(row['translated']):
        return None
    for phy_nm in phy_nm_list:
        if similarity_percentage(row['translated'], phy_nm.upper()) > 80:
            return phy_nm
    return None

# List of PHY_NM to match with translated names
phy_nm_list = df['PHY_NM'].dropna().tolist()

# Apply the function to find matched names
df['matched_name'] = df.apply(lambda row: find_matched_name(row, phy_nm_list), axis=1)

# Save the updated dataframe to a new Excel file
output_file_path = '/kaggle/working/translated_and_matched_names.xlsx'
df.to_excel(output_file_path, index=False)

# Display the first few rows of the updated dataframe
df.head()


In [ ]:
import pandas as pd

# Load the Excel files
file1_path = '/kaggle/input/matched-name/translated_and_matched_names (4).xlsx'
file2_path = '/kaggle/input/physician-database/Physician Database 500 PRS.xlsx'

df1 = pd.read_excel(file1_path)
df2 = pd.read_excel(file2_path)

# Display the columns to verify names
print("Columns in df1:", df1.columns)
print("Columns in df2:", df2.columns)

# Assuming the relevant columns
# df1 has columns: 'matched_name', 'other_columns...'
# df2 has columns: 'PHY_NM' for physician name and 'PHY_ID' for physician ID

# Create a dictionary for quick lookup from physician database
phy_id_dict = dict(zip(df2['PHY_NM'].str.upper(), df2['PHY_ID']))

# Function to find physician ID based on matched name
def find_physician_id(matched_name):
    if pd.isnull(matched_name):
        return None
    return phy_id_dict.get(matched_name.upper(), None)

# Apply the function to create a new column 'extracted_phy_ID'
df1['extracted_phy_ID'] = df1['matched_name'].apply(find_physician_id)

# Save the updated dataframe to a new Excel file
output_file_path = '/kaggle/working/updated_translated_and_matched_names.xlsx'
df1.to_excel(output_file_path, index=False)

# Display the first few rows of the updated dataframe
df1.head()